In [1]:
import requests
import re
import os
import zipfile

# Set your personal access token, project ID, and pipeline ID here
access_token = '<access_token'
project_id = '140223'
pipeline_id = '<pipeline_id>'

 # Create the 'artifacts' folder if it doesn't exist
if not os.path.exists('artifacts'):
    os.makedirs('artifacts')


# This is the base URL for GitLab API
base_url = f"https://gitlab.lrz.de/api/v4/projects/{project_id}/pipelines/{pipeline_id}"
headers = {'Private-Token': access_token}

# Function to format name
def format_name(name):
    return name.replace("_", "-").replace(": [", "_").replace(", ", "_").replace("]", "")
#    return re.sub(r'[^A-Za-z0-9_\-]', '', name.title())

# Function to get all jobs from GitLab with pagination
def get_all_jobs():
    jobs = []
    page = 1
    while True:
        response = requests.get(f"{base_url}/jobs?scope[]=success&per_page=10&page={page}", headers=headers)
        if response.status_code != 200:
            print(f"Request failed with status {response.status_code}")
            break
        new_jobs = response.json()
        if not new_jobs:  # if there are no more jobs, stop
            break
        jobs.extend(new_jobs)
        page += 1
    return jobs

jobs = get_all_jobs()

for job in jobs:
    if job['stage'] != "evaluate":
      print(f"Skipping job:{job['name']}")
      continue

    job_id = job['id']
    job_name = format_name(job['name'])

    artifact_url = f"https://gitlab.lrz.de/api/v4/projects/{project_id}/jobs/{job_id}/artifacts"
    response = requests.get(artifact_url, headers=headers, stream=True)

    if response.status_code != 200:
        print(f"Failed to get artifacts for job {job_id} with status {response.status_code}")
    else:
        filename = f"artifacts/{job_name}.zip"
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Saved artifacts for job {job_id} as {filename}")

Saved artifacts for job 6380143 as artifacts/n8n_io_high.zip
Saved artifacts for job 6375991 as artifacts/n8n_ram_medium.zip
Skipping job:n8n_build
Saved artifacts for job 6375568 as artifacts/FreeCodeCamp_none_none.zip
Saved artifacts for job 6375566 as artifacts/Mermaid_io_medium.zip
Saved artifacts for job 6371010 as artifacts/FreeCodeCamp_io_low.zip
Saved artifacts for job 6364094 as artifacts/Mermaid_netbandwidth_high.zip
Saved artifacts for job 6364093 as artifacts/Mermaid_netbandwidth_medium.zip
Saved artifacts for job 6364092 as artifacts/Mermaid_netbandwidth_low.zip
Saved artifacts for job 6364091 as artifacts/Mermaid_netpgk_high.zip
Saved artifacts for job 6364090 as artifacts/Mermaid_netpgk_medium.zip
Saved artifacts for job 6364089 as artifacts/Mermaid_netpgk_low.zip
Saved artifacts for job 6364088 as artifacts/Mermaid_netdelay_high.zip
Saved artifacts for job 6364087 as artifacts/Mermaid_netdelay_medium.zip
Saved artifacts for job 6364086 as artifacts/Mermaid_netdelay_low.

In [2]:

import zipfile
import os
from pathlib import Path

# Create the 'json-results' folder if it doesn't exist
if not os.path.exists('sorted-results'):
    os.makedirs('sorted-results')

# Get a list of all zip files in the 'artifacts' folder
zip_files = [file for file in os.listdir('artifacts') if file.endswith('.zip')]
# Delete 'artifactsBuildPackage.zip' if it exists
if 'BuildPackage.zip' in zip_files:
    zip_files.remove('BuildPackage.zip')
    os.remove(os.path.join('artifacts', 'BuildPackage.zip'))

print("zip files: ", zip_files)

# Extract aggregated_test_results.json from each zip file
for zip_file in zip_files:
    zip_path = os.path.join('artifacts', zip_file)

    # Split the filename
    parts = zip_file.replace(".zip", "").split("_")
    # Assign parts to variables
    projname, noise_type, noise_level = parts[0], parts[1], parts[2]

    sorted_path = f"sorted-results/{projname}/{noise_type}-{noise_level}"
    os.makedirs(sorted_path, exist_ok=True)

    # Extract the inner experiment.zip file
    with zipfile.ZipFile(zip_path, 'r') as outer_zip:

        print("file name: ", outer_zip)
        inner_zip_path = 'experiment.zip'
        outer_zip.extract(inner_zip_path, path='temp')  # Extract to a temporary folder

    # Extract everythin from inner zip file
    inner_zip_path = os.path.join('temp', inner_zip_path)
    with zipfile.ZipFile(inner_zip_path, 'r') as inner_zip:
        #print(inner_zip.namelist())
        inner_zip.extractall(path=sorted_path)

    src_path = f"{sorted_path}/aggregate-test-results"

    for each_file in Path(src_path).glob('*'): # grabs all files
        trg_path = each_file.parent.parent # gets the parent of the folder
        each_file.rename(trg_path.joinpath(each_file.name)) # moves to parent folder.
    os.rmdir(src_path)
    # Clean up the temporary folder
    os.remove(inner_zip_path)
    os.rmdir('temp')

print("Extraction complete.")

zip files:  ['n8n_netpgk_low.zip', 'n8n_netdelay_low.zip', 'n8n_netbandwidth_medium.zip', 'n8n_ram_low.zip', 'Mermaid_cpu_low.zip', 'n8n_io_medium.zip', 'FreeCodeCamp_netdelay_high.zip', 'n8n_io_high.zip', 'FreeCodeCamp_io_high.zip', 'FreeCodeCamp_netpgk_high.zip', 'n8n_netbandwidth_low.zip', 'n8n_netpgk_high.zip', 'n8n_ram_high.zip', 'FreeCodeCamp_io_low.zip', 'Mermaid_netbandwidth_high.zip', 'n8n_netpgk_medium.zip', 'Mermaid_netbandwidth_low.zip', 'FreeCodeCamp_netdelay_low.zip', 'Mermaid_netdelay_low.zip', 'FreeCodeCamp_netpgk_medium.zip', 'Mermaid_netpgk_high.zip', 'n8n_none_none.zip', 'FreeCodeCamp_none_none.zip', 'Mermaid_io_low.zip', 'Mermaid_netdelay_high.zip', 'Mermaid_cpu_medium.zip', 'FreeCodeCamp_cpu_low.zip', 'n8n_netdelay_medium.zip', 'FreeCodeCamp_io_medium.zip', 'Mermaid_ram_medium.zip', 'Mermaid_ram_low.zip', 'FreeCodeCamp_netbandwidth_low.zip', 'n8n_io_low.zip', 'Mermaid_io_high.zip', 'FreeCodeCamp_cpu_medium.zip', 'FreeCodeCamp_netbandwidth_medium.zip', 'FreeCodeCamp

In [3]:

# Create a zip file of the 'json-results' folder
zip_name = 'sorted-results.zip'
os.system(f"zip -r {zip_name} sorted-results")

# Download the zip file
from google.colab import files
files.download(zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cleanup folders

In [ ]:

import shutil
import os
shutil.rmtree("sorted-results", ignore_errors=True)
shutil.rmtree("artifacts", ignore_errors=True)
try:
    os.remove("sorted-results.zip")
except OSError as error:
  pass
